In [28]:
from configparser import ConfigParser
import datetime
import pandas as pd
import pyodbc

In [29]:

CurrentDate = str(datetime.datetime.now())

delimiter1 = " ";
delimiter2 = "-";
CurrentDate = CurrentDate.replace(delimiter2, delimiter1);
CD = CurrentDate.split(delimiter1);
date = CD[2]
month = CD[1]
year = CD[0]


In [30]:
CD

['2023', '02', '22', '06:24:15.761226']

In [31]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
# conn.autocommit = False
cursor = conn.cursor()

In [32]:
sql_delete_query1  = "delete from FINCORE_DB.Fincore_Analytics.RISK_PERSON"
sql_delete_query2  = "delete from FINCORE_DB.Fincore_Analytics.RISK_PERSON_ANALYTICAL_RULES"
sql_delete_query3  = "delete from FINCORE_DB.Fincore_Analytics.RISK_LKP_ENTITY_THRESHOLD where RULE_NO like 'P0%'"
sql_delete_query4  = "delete from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES"
sql_delete_query5  = "delete from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG"
sql_delete_query6  = "delete from FINCORE_DB.Fincore_Analytics.RISK_PERSON_INCREMENTAL"
sql_delete_query7  = "delete from FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_PEER_GRP_ATTRIBUTES WHERE ENTITY_TYPE = 'I' "


cursor.execute(sql_delete_query1)
cursor.execute(sql_delete_query2)
cursor.execute(sql_delete_query3)
cursor.execute(sql_delete_query4)
cursor.execute(sql_delete_query5)
cursor.execute(sql_delete_query6)
cursor.execute(sql_delete_query7)
conn.commit()

In [33]:

sql_delete_query3  = "delete from FINCORE_DB.Fincore_Analytics.RISK_LKP_ENTITY_THRESHOLD where RULE_NO like 'P0%'"

cursor.execute(sql_delete_query3)

conn.commit()

In [34]:
count_of_audit_ingt = pd.read_sql_query("select count(1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN where JOB_NAME = '2_RISK_PERSON' and JOB_STATUS = 'Success'",conn)

In [35]:
count_of_audit_ingt = count_of_audit_ingt.iloc[0,0]

In [36]:
count_of_audit_ingt

30

In [37]:
try:
    
#     if ((date == '20' and month == '02') or ((date == '01' and month == '06')) or (count_of_audit_ingt == 0)):
    if ((date == '80' and month == '02') or ((date == '01' and month == '06')) or (count_of_audit_ingt == 0) or (year == '2023')):

        
        sql_delete_query1  = "delete from FINCORE_DB.Fincore_Analytics.RISK_PERSON"
        sql_delete_query2  = "delete from FINCORE_DB.Fincore_Analytics.RISK_PERSON_ANALYTICAL_RULES"
        sql_delete_query3  = "delete from FINCORE_DB.Fincore_Analytics.RISK_LKP_ENTITY_THRESHOLD where RULE_NO like 'P0%'"
        sql_delete_query4  = "delete from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES"
        sql_delete_query5  = "delete from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG"
        sql_delete_query6  = "delete from FINCORE_DB.Fincore_Analytics.RISK_PERSON_INCREMENTAL"
        sql_delete_query7  = "delete from FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_PEER_GRP_ATTRIBUTES"
        
        
        cursor.execute(sql_delete_query1)
        cursor.execute(sql_delete_query2)
        cursor.execute(sql_delete_query3)
        cursor.execute(sql_delete_query4)
        cursor.execute(sql_delete_query5)
        cursor.execute(sql_delete_query6)
        cursor.execute(sql_delete_query7)
        
        conn.commit()
        


        %run master_data_pop_all_data.ipynb
        %run PEER_GROUPING.ipynb
        %run Threshold_Computation_V2.ipynb
        %run Entity_Risk_Calculation_V2_1.ipynb
        %run Entity_calculation_2.ipynb
        

        

    else:
        print("Else")
        %run master_data_pop.ipynb
        %run Entity_Risk_V2_Incremental_data.ipynb
        %run Threshold_Computation_V2_Incremental_data.ipynb
        %run Entity_Risk_Calculation_V2_1_Incremental_data.ipynb
        

    
    
    
except Exception as error:
    %store -r job_id
    error = str(error)
    print("Here error")
    error = error.replace('\'','')
    error = error.replace('\"','')
    print("Failed to update record to database rollback: {}".format(error))
    conn.rollback()
    conn.execute(f'''update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set ERROR_DESC = '{error}'  where  JOB_RUN_ID = {job_id}''')
    conn.commit()
    conn.close()
    

Stored 'job_id' (int64)
master_data_pop_per_q1  DONE!
master_data_pop_per_q2  DONE!
per_pincode_update  DONE!
per_occ_update  DONE!
master_data_pop_per_income  DONE!
per_trx_update_credit  DONE!
per_trx_update_debit  DONE!
peer_grouping_columns  DONE!
MOBILE_COUNT  DONE!
TELEPHONE_COUNT  DONE!
EMAIL_COUNT  DONE!
WATCHLIST_COUNT  DONE!
STR_REPORT_COUNT  DONE!
CTR_REPORT_COUNT  DONE!
NTR_REPORT_COUNT  DONE!
PTR_COUNT  DONE!
CBWTR_REPORT_COUNT  DONE!
CCR_REPORT_COUNT  DONE!
str_amount_debit  DONE!
str_amount_credit  DONE!
ctr_amount_debit  DONE!
ctr_amount_credit  DONE!
ntr_amount_debit  DONE!
ntr_amount_credit  DONE!
cbwtr_amount_debit  DONE!
cbwtr_amount_credit  DONE!
ccr_amount_debit  DONE!
ccr_amount_credit  DONE!
('\r\n'
 '            MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_PERSON_Summary_stg_RULES \r\n'
 '\t\t\tWITH (HOLDLOCK) as target  USING   (   \r\n'
 '\t\t\tSELECT   PMCFR.Person_Master_Id,      \r\n'
 '\t\t\tSUM( 1 ) AS CBWTR_COUNT   \r\n'
 '\t\t\tFROM [FINCORE_DB].[F

100%|██████████| 5/5 [00:00<00:00, 70.93it/s]


Working on  Government Sector 1
lenX 113
Value Error
Clustering done for 1
It took 0:00:00.000003 hours to cluster Government Sector in 1 clusters
2023-02-22 06:24:21.959646
Working on  Government Sector 2
lenX 113
Clustering done for 2
It took 0:00:00.000003 hours to cluster Government Sector in 2 clusters
2023-02-22 06:24:21.971850
Working on  Government Sector 3
lenX 113
Clustering done for 3
It took 0:00:00.000004 hours to cluster Government Sector in 3 clusters
2023-02-22 06:24:21.986634
Working on  Government Sector 4
lenX 113
Clustering done for 4
It took 0:00:00.000005 hours to cluster Government Sector in 4 clusters
2023-02-22 06:24:22.003043
Working on  Government Sector 5
lenX 113
Clustering done for 5
It took 0:00:00.000005 hours to cluster Government Sector in 5 clusters
2023-02-22 06:24:22.019614
Clustering for Profession Self Employed


100%|██████████| 5/5 [00:00<00:00, 70.12it/s]


Working on  Self Employed 1
lenX 18
Value Error
Clustering done for 1
It took 0:00:00.000002 hours to cluster Self Employed in 1 clusters
2023-02-22 06:24:22.029628
Working on  Self Employed 2
lenX 18
Clustering done for 2
It took 0:00:00.000003 hours to cluster Self Employed in 2 clusters
2023-02-22 06:24:22.041978
Working on  Self Employed 3
lenX 18
Clustering done for 3
It took 0:00:00.000004 hours to cluster Self Employed in 3 clusters
2023-02-22 06:24:22.056764
Working on  Self Employed 4
lenX 18
Clustering done for 4
It took 0:00:00.000005 hours to cluster Self Employed in 4 clusters
2023-02-22 06:24:22.073561
Working on  Self Employed 5
lenX 18
Clustering done for 5
It took 0:00:00.000005 hours to cluster Self Employed in 5 clusters
2023-02-22 06:24:22.092405
Clustering for Profession Others


...

257it [00:00, 841.58it/s]

[1.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 48.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 20.0, 50.0, 40.0, 12.0, 12.0, 'P002001013_P003001004_P003001007_P003001008_P003001009_P003001010', 2.935483870967742]
[2.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 48.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 20.0, 50.0, 40.0, nan, nan, 'P002001013_P003001004_P003001007_P003001008', 3.2916666666666665]
[3.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 48.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 20.0, 50.0, 40.0, nan, nan, 'P002001013_P003001004_P003001007_P003001008', 3.2916666666666665]
[4.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 48.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 20.0, 50.0, 40.0, nan, nan, 'P002001013_P003001004_P003001007_P003001008', 3.2916666666666665]
[5.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 48.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 20.0, 50.0, 40.0, nan, nan, 'P0020010

349it [00:00, 857.74it/s]


[324.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 48.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 20.0, nan, 40.0, 28.0, 28.0, 'P002001013_P003001004_P003001008_P003001009_P003001010', 2.342857142857143]
[325.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 48.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 20.0, nan, 40.0, 28.0, 28.0, 'P002001013_P003001004_P003001008_P003001009_P003001010', 2.342857142857143]
[326.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '', 0.0]
[327.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '', 0.0]
[328.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '', 0.0]
[329.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 48.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, 24.0, nan,

349it [00:00, 847.30it/s]
